# Zipcode Choropleth with Plotly, Pandas and Mapbox

In this tutorial, we will be creating a choropleth of San Diego's Zipcodes based on Zillow's Price-to-Rent Ratio.


#### Mapbox Access Token

To plot on Mapbox maps with Plotly you'll need a Mapbox account and a [Mapbox Access Token](https://www.mapbox.com/studio) which you can add to your [Plotly settings](https://plot.ly/settings/mapbox). If you're using a Chart Studio Enterprise server, please see additional instructions here: https://help.plot.ly/mapbox-atlas/. 

#### Read the Two Data Sets

In [8]:
import plotly
import pandas as pd
import string
import urllib

ratio_url = 'file:///Users/brian/Zillow/sdZip_PriceToRentRatio_AllHomes.csv'
geo_data_url = 'file:///Users/brian/Zillow/SanDiegoZipCodes.geojson'

ratio_data = pd.read_csv(ratio_url)
ratio_data.columns = ['RegionName','City', 'State','CountyName','Ratio']

geo_data = pd.read_json(geo_data_url)


#### Bucket Data, Create stub GeoJson lists, Convert to Standared GeoJson

In [5]:
# Define the buckets
green_zips = ratio_data[ratio_data.Ratio <= 13]
olive_zips = ratio_data[(ratio_data.Ratio > 13) & (ratio_data.Ratio < 15)]
yellow_zips = ratio_data[(ratio_data.Ratio > 15) & (ratio_data.Ratio < 17)]
orange_zips = ratio_data[(ratio_data.Ratio > 17) & (ratio_data.Ratio < 19)]
red_zips = ratio_data[ratio_data.Ratio >= 19]

# Define the stub lists that will contain the geo data (note this isn't in proper GeoJson format yet)
green_geo = []
olive_geo = []
yellow_geo = []
orange_geo = []
red_geo = []

# Create bucketed lists
entryIndex=-1
for entry in geo_data['features']:
    entryIndex+=1
    intZip = int(entry['properties']['zip'])
    # Ensure bucket is not empty, and we have a "hit" for the entry zip in the bucket
    if len(green_zips.index) > 0 and len(green_zips[green_zips.RegionName == intZip]) > 0:
        print("💚"),
        green_geo.append(geo_data['features'][entryIndex])
    elif len(olive_zips.index) > 0 and len(olive_zips[olive_zips.RegionName == intZip]) > 0:
        print("🍸"),
        olive_geo.append(geo_data['features'][entryIndex])
    elif len(yellow_zips.index) > 0 and len(yellow_zips[yellow_zips.RegionName == intZip]) > 0:
        print("💛"),
        yellow_geo.append(geo_data['features'][entryIndex])
    elif len(orange_zips.index) > 0 and len(orange_zips[orange_zips.RegionName == intZip]) > 0:
        print("🧡 "),
        orange_geo.append(geo_data['features'][entryIndex])
    elif len(red_zips.index) > 0 and len(red_zips[red_zips.RegionName == intZip]) > 0:
        print("❤️ "),
        red_geo.append(geo_data['features'][entryIndex])
    else:
        print("."),

import json
# Creating GeoJson standard files
green_data = {"type": "FeatureCollection"}
green_data['features'] = green_geo

olive_data = {"type": "FeatureCollection"}
olive_data['features'] = olive_geo

yellow_data = {"type": "FeatureCollection"}
yellow_data['features'] = yellow_geo

orange_data = {"type": "FeatureCollection"}
orange_data['features'] = orange_geo

red_data = {"type": "FeatureCollection"}
red_data['features'] = red_geo

# Writing the new GeoJson files out (which we'll add/commit/push to GitHub)
with open('sd-green-data.json', 'w') as f:
    f.write(json.dumps(green_data))
    
with open('sd-olive-data.json', 'w') as f:
    f.write(json.dumps(olive_data))
    
with open('sd-yellow-data.json', 'w') as f:
    f.write(json.dumps(yellow_data))
    
with open('sd-orange-data.json', 'w') as f:
    f.write(json.dumps(orange_data))
    
with open('sd-red-data.json', 'w') as f:
    f.write(json.dumps(red_data))

❤️  ❤️  . 💚 🧡  🧡  ❤️  ❤️  . 🧡  🧡  🧡  ❤️  🧡  💛 . ❤️  ❤️  🧡  💛 . . ❤️  . 💛 💛 🧡  🧡  🧡  . 💛 🧡  . ❤️  💛 💚 ❤️  ❤️  ❤️  ❤️  ❤️  ❤️  🧡  🧡  💛 ❤️  🧡  🧡  ❤️  ❤️  🧡  ❤️  . 🧡  🧡  🧡  🧡  🧡  . . 🧡  ❤️  🧡  💛 💛 🧡  . ❤️  . . ❤️  🧡  . 🧡  ❤️  🧡  🧡  ❤️  . . . 🧡  🧡  ❤️  ❤️  ❤️  ❤️  💛 ❤️  ❤️  💛 ❤️  🧡  ❤️  💛 💛 🧡  ❤️  ❤️  ❤️  ❤️  ❤️  ❤️  . 💛 ❤️  . . 🧡  ❤️  ❤️  ❤️  ❤️  ❤️  ❤️  . . 💛 . . . .


#### Choropleth of Zillow Price to Rent Ratio by San Diego County Zipcode

Make sure to have a [Mapbox Access Tolken](https://www.mapbox.com/studio) for the generation of the plot. For more information on plotting Mapbox maps in Python, checkout the [documentation](https://plot.ly/python/scattermapbox/) 

In [3]:
import plotly.plotly as py
import plotly.graph_objs as graph_objs

mapbox_access_token = "ENTER_YOUR_MAPBOX_TOKEN_HERE"

data = graph_objs.Data([
    graph_objs.Scattermapbox(
        lat=['32.7157'],
        lon=['-117.1611'],
        mode='markers'
    )
])
layout = graph_objs.Layout(
    title='Choropleth of Zillow Price to Rent Ratio by San Diego County Zipcode',
    height=600,
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        layers=[
            dict(
                sourcetype = 'geojson',
                source = 'https://raw.githubusercontent.com/brianjester/choropleth/master/SanDiegoZipCodes.geojson',
                type = 'line',
                color = 'rgba(128,128,128,0.8)'
            ),
            dict(
                sourcetype = 'geojson',
                source = 'https://raw.githubusercontent.com/brianjester/choropleth/master/sd-green-data.json',
                type = 'fill',
                color = 'rgba(0,255,0,0.8)'
            ),
            dict(
                sourcetype = 'geojson',
                source = 'https://raw.githubusercontent.com/brianjester/choropleth/master/sd-olive-data.json',
                type = 'fill',
                color = 'rgba(191,255,0,0.8)'
            ),
            dict(
                sourcetype = 'geojson',
                source = 'https://raw.githubusercontent.com/brianjester/choropleth/master/sd-yellow-data.json',
                type = 'fill',
                color = 'rgba(255,255,0,0.8)'
            ),
            dict(
                sourcetype = 'geojson',
                source = 'https://raw.githubusercontent.com/brianjester/choropleth/master/sd-orange-data.json',
                type = 'fill',
                color = 'rgba(255,128,0,0.8)'
            ),
            dict(
                sourcetype = 'geojson',
                source = 'https://raw.githubusercontent.com/brianjester/choropleth/master/sd-red-data.json',
                type = 'fill',
                color = 'rgba(255,0,0,0.8)'
            )
        ],
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=32.7157,
            lon=-117.1611
        ),
        pitch=0,
        zoom=7,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='zip-level-choropleths-python')



/Users/brian/anaconda2/lib/python2.7/site-packages/plotly/graph_objs/_deprecations.py:39: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.




Suggested improvements: Add a legend, add Zip code numbers to the map